In [1]:
import glob
import os
import pandas as pd
import argparse
import subprocess
import re
from IPython.core import display as ICD
from os import listdir
from multiprocessing import Pool

from shutil import rmtree
from subprocess import check_output

# Chocolate Milkshake Betaseries 


## Table of Conents:

* [FSL feat1 analysis](#second-bullet)  
* [Concatenate with fslmerge](#third-bullet)
* [Transform functionals with flirt and fslmath](#fourth-bullet)
* [Pull individual ROI timeseries by subject](#fifth-bullet)


**The Paradigm**

Functional Tasks: 

    milkshake pic, milkshake receipt, h20 pic, h20 receipt, milkshake minus h20, milkshake plus h20, rinse

Furthermore, `milkshake` is broken down into the following specifities:   
      
    HF(high fat, HS(high sugar), and respectively, LF(low fat), LS(low sugar)   

with the following relationships:  
    
    HF_HS, HF_LS, LF_HS, LF_LS



In [2]:
# view the onsets available
!ls /projects/niblab/experiments/chocolate_milkshake/data/onsets

GNG  GNG_wrong	imagine  milkshake


In [3]:
print('[INFO] EV path: {}'.format('/projects/niblab/experiments/chocolate_milkshake/data/onsets/milkshake'))



[INFO] EV path: /projects/niblab/experiments/chocolate_milkshake/data/onsets/milkshake


In [99]:
# view directory contents
#listdir('/projects/niblab/experiments/chocolate_milkshake/data/onsets/milkshake')


In [5]:

def chunks(l,n):
    return [l[i:i+n] for i in range(0, len(l), n)]


## FSL feat1 analysis <a class='anchor' id='second-bullet'></a>

### Load and set variables

In [6]:
choco_evs=glob.glob('/projects/niblab/data/eric_data/ev_files/milkshake/*.ev')
choco_evs=[x.split('/')[-1].replace('.ev',"") for x in choco_evs]
choco_evs=[re.sub('mk[A-Z]_', '',x) for x in choco_evs]
choco_evs=set(choco_evs)
print("[INFO] set of unique chocolate evs: \n", choco_evs)

[INFO] set of unique chocolate evs: 
 {'h20_pic', 'h20_receipt', 'HF_HS_minus_h20', 'HF_LS_receipt', 'LF_LS_receipt', 'rinse', 'HS_plus_h20', 'HF_HS_receipt', 'LF_HS_receipt', 'LF_HS_minus_h20', 'milkshake_pic'}


In [50]:
for ev in choco_evs:
    if "receipt" in ev:
        pass
    else:
        print(ev)

h20_pic
HF_HS_minus_h20
rinse
HS_plus_h20
LF_HS_minus_h20
milkshake_pic


### Make subject condition feat files (`.fsf`)

In [94]:
def make_file(sub_id, ses_id, trial_id, output_dir, task,data_dict):
    with open(os.path.join('/projects/niblab/experiments/chocolate_milkshake/data/code/beta_design.fsf'),'r') as infile:
        tempfsf=infile.read()
        #
        if not os.path.exists(os.path.join(output_dir, "design_files")):
            os.makedirs(os.path.join(output_dir, "design_files"))
        #print(output_dir)
        design_fileout = os.path.join(output_dir, "design_files/%s_%s_%s_feat1.fsf"%(sub_id, ses_id, trial_id))
        out_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["OUTPUT"]
        func_param = data_dict[sub_id][task]["FUNCRUN"]
        confound_param = data_dict[sub_id][task]["CONFOUND"]
        trial_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["TRIAL"]
        nuis_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["NUIS"]

        tempfsf = tempfsf.replace("OUTPUT", out_param)
        tempfsf = tempfsf.replace("FUNCRUN", func_param) 
        tempfsf = tempfsf.replace("CONFOUND", confound_param)
        tempfsf = tempfsf.replace("TRIAL", trial_param)
        tempfsf = tempfsf.replace("NUIS", nuis_param)

        for i in range(6):
            moco = data_dict[sub_id][task]["MOCO%i"%i]
            tempfsf = tempfsf.replace("MOCO%i"%i, moco)
        try:
            with open(design_fileout,'w') as outfile:
                outfile.write(tempfsf)
            outfile.close()
        except:
            print("BAD SUBJECT ", sub_id)
        infile.close()

        
        

In [99]:
def create_fsf(input_dir, deriv_path, ses_id):
    orig_path='/projects/niblab/bids_projects/Experiments/ChocoData/derivatives'
    ses_id=ses_id
    data_dict= {}
    # start loop -- looping through subjects
    subject_list = glob.glob(os.path.join(deriv_path, 'preprocessed', 'sub-*/%s'%ses_id))
    for sub_path in sorted(subject_list):
        #print(sub_path)
        sub_id=sub_path.split("/")[-2]
        functional_tasks = glob.glob(os.path.join(orig_path,'%s/ses-1'%sub_id, 'func/*milkshake*preproc_brain.nii.gz'))
        if sub_id not in data_dict:
            data_dict[sub_id] = {}
                
        
        for functional in functional_tasks: # SECOND LOOP -- looping through RUNS
            #print(sub_id)
            task=functional.split("/")[-1].split("_")[2].split("-")[1]
            #print(task)
            analysis_folder=os.path.join(deriv_path, 'preprocessed/%s/%s'%(sub_id,ses_id), "analysis")
            #print(analysis_folder)
            output_dir = os.path.join(analysis_folder, 'beta/task-%s'%task)
            
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            
            confound_file = os.path.join(orig_path, "%s/ses-1"%sub_id, "func/motion_assessment/%s_%s_task-%s_bold_space-MNI152NLin2009cAsym_preproc_brain_confound.txt"%(sub_id,ses_id, task))
                
            data_dict[sub_id][task] = {
                "TRIALS" : { },
                "CONFOUND" : confound_file,
                "FUNCRUN" : functional
              }           
            
            for i in range(6):
                motcor=os.path.join(orig_path, "%s/ses-1"%sub_id, 'func','motion_assessment', 'motion_parameters','%s_%s_task-%s_moco%s.txt'%(sub_id,ses_id, task,i))
                data_dict[sub_id][task]['MOCO%i'%i] = motcor
            
            
            subj_trials = sorted(glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/evs/trials/%s*trial*.txt'%task.replace('milkshake', 'mk')))
            
            #print(subj_trials)
            
            
            if not subj_trials:
                pass
            else:
                for trial_file in subj_trials:
                    _id = sub_id.split("-")[1]
                    _id = _id[1:]
                    trial_id = trial_file.split("/")[-1].split(".")[0]
                    #print(trial_id)
                    nuis_file = os.path.join('/projects/niblab/experiments/chocolate_milkshake/data/evs/nuis', '%s.txt'%trial_id.replace('trial', 'nuis'))
                    #print(nuis_file)
                    fileout = os.path.join(output_dir, "%s_%s_%s"%(sub_id, ses_id,trial_id))
                    #print(fileout)
                    data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id] = {"TRIAL" : trial_file, "NUIS": nuis_file, "OUTPUT" : fileout}
                    make_file(sub_id, ses_id, trial_id,output_dir, task, data_dict)
                    
    print("[INFO] completed making fsl feat1 files.")               
    return(data_dict)



In [100]:
def fsl_feat1():
    deriv_path='/projects/niblab/experiments/chocolate_milkshake/data/bids/derivatives'
    ses_id='ses-1'
    input_dir = '/projects/niblab/experiments/chocolate_milkshake'
    data_dict=create_fsf(input_dir, deriv_path,ses_id)
    
fsl_feat1()

[INFO] completed making fsl feat1 files.


**Run Slurm Jobs**

In [145]:
subject_list =  [x.split("/")[-2].split("-")[1].strip('0') for x in glob.glob(os.path.join('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives', 'sub-*/ses-1'))]
subject_list[:5]

['1', '4', '5', '9', '11']

In [103]:
def slurm_by_file(subject,keyword):
    fsfs=glob.glob(os.path.join('/projects/niblab/experiments/chocolate_milkshake/data/bids/derivatives/preprocessed/{}/ses-1/analysis/beta/task*/design_files/*{}*.fsf'.format(subject, keyword)))
    for fsf in fsfs:
        slurm_cmd = "sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job {}".format(fsf)
        #print('[INFO] submitted: \n', slurm_cmd)
        os.system(slurm_cmd)
    
        

Submit slurm jobs by running fsf files individually:

In [104]:
subject_ids=[x.split('/')[-2] for x in glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/bids/derivatives/preprocessed/*/ses-1')]
keyword="pic"
for subject in subject_ids:
    #print(subject)
    slurm_by_file(subject, keyword)

print('[INFO] submitted slurm jobs.')
    

[INFO] submitted slurm jobs.


In [105]:
!squeue -u nbytes


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           3078091     batch betaseri   nbytes PD       0:00      1 (Priority)
           3078093     batch betaseri   nbytes PD       0:00      1 (Priority)
           3078094     batch betaseri   nbytes PD       0:00      1 (Priority)
           3078096     batch betaseri   nbytes PD       0:00      1 (Priority)
           3078097     batch betaseri   nbytes PD       0:00      1 (Priority)
           3078098     batch betaseri   nbytes PD       0:00      1 (Priority)
           3078099     batch betaseri   nbytes PD       0:00      1 (Priority)
           3078100     batch betaseri   nbytes PD       0:00      1 (Priority)
           3078101     batch betaseri   nbytes PD       0:00      1 (Priority)
           3078102     batch betaseri   nbytes PD       0:00      1 (Priority)
           3078103     batch betaseri   nbytes PD       0:00      1 (Priority)
           3078104     batch betaseri   nbytes

           3076522     batch betaseri   nbytes  R      14:16      1 compute-7-30
           3076523     batch betaseri   nbytes  R      14:16      1 compute-1-1
           3076524     batch betaseri   nbytes  R      14:16      1 compute-1-1
           3076525     batch betaseri   nbytes  R      14:16      1 compute-1-1
           3076526     batch betaseri   nbytes  R      14:16      1 compute-1-1
           3076527     batch betaseri   nbytes  R      14:16      1 compute-1-1
           3076528     batch betaseri   nbytes  R      14:16      1 compute-1-31
           3076530     batch betaseri   nbytes  R      14:16      1 compute-1-31
           3076531     batch betaseri   nbytes  R      14:16      1 compute-1-31
           3076532     batch betaseri   nbytes  R      14:16      1 compute-1-31
           3076533     batch betaseri   nbytes  R      14:16      1 compute-1-31
           3076534     batch betaseri   nbytes  R      14:16      1 compute-1-24
           3076535     batch beta

In [28]:
#listdir('/projects/niblab/experiments/chocolate_milkshake/data/bids/derivatives/preprocessed/sub-113/ses-1/analysis/beta/task-milkshakeB')



## Add feat directories to data dictionary

In [29]:
# load and set variables
beta_path='/projects/niblab/experiments/chocolate_milkshake/data/betaseries'
concat_path="/projects/niblab/experiments/chocolate_milkshake/data/betaseries/niftis_concat"
subject_folders=sorted(glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/bids/derivatives/preprocessed/sub-*'))

data_dict={}
bad_subs=[]

for folder in subject_folders:
    subject_id=folder.split("/")[-1]
    #print(os.path.join(folder,'ses-1/analysis/beta/*.feat'))
    tasks=glob.glob(os.path.join(folder,'ses-1/analysis/beta/*'))
    #print(tasks)
    if not tasks:
        bad_subs.append(subject_id)
    else:
        if subject_id not in data_dict:
            data_dict[subject_id]={}
        for task_folder in tasks:
            #print(task_folder)
            task=task_folder.split("/")[-1].split('.')[0]
            #print(task)
            if task not in data_dict[subject_id]:
                data_dict[subject_id][task]={}
            pes=glob.glob(os.path.join(task_folder, '*.feat'))
            data_dict[subject_id][task]=pes
       
    
    

In [38]:
for subject in data_dict:
    #print("[INFO] ", subject)
    for task in data_dict[subject]:
        #print("[INFO] ", task)
        dir_ct=0
        
        
        for directory in data_dict[subject][task]:
            
            if "minus" in directory:
                dir_ct+=1
        if dir_ct < 9:
            print("[INFO] %s %s"%( subject, task))
            print(dir_ct)
        #print(dir_ct)
            

## Concatenate (fslmerge) data <a class='anchor' id='third-bullet'></a>  
`fslmerge : concatente image files into a single output.`   
  
In this step we use the `fslmerge` command to combine the `pe.nii.gz` files by subject of the same condition.  




In [64]:
beta_path='/projects/niblab/experiments/chocolate_milkshake/data/betaseries'
concat_path="/projects/niblab/experiments/chocolate_milkshake/data/betaseries/niftis_concat"
keyword="LF_HS_minus_h20"


In [68]:

def merge_files(subject_list, data_dict=data_dict,keyword=keyword):

    for subject_id in subject_list:
        for task in data_dict[subject_id]:
            #print(task)
            feat_path='/projects/niblab/experiments/chocolate_milkshake/data/bids/derivatives/preprocessed/%s/ses-1/analysis/beta/%s/*%s*.feat/stats/pe1.nii.gz'%(subject_id, task, keyword)
            pes=glob.glob(os.path.join(feat_path))
            #print(pes)
            #print(trial)
            #filename=trial.replace(subject_id+"_", "")
            extension=task+"_%s"%keyword
            outfile = os.path.join(concat_path, "%s/%s_%s"%(keyword, subject_id, extension))
            #print(outfile)
            if not pes:
                bad_subs.append(subject_id)
            else:
                pe_str = " ".join(pes)
                fslmerge_cmd="/projects/niblab/modules/software/fsl/5.0.10/bin/fslmerge -t %s %s"%(outfile, pe_str)
                #print("[INFO] running fsl merge command...")
                #print(fslmerge_cmd, "\n")
                #os.system(fslmerge_cmd)


            

In [67]:
chunksize=10
print("[INFO] chunksize: {}".format(chunksize))
chunk_list=chunks(subject_ids, chunksize)

#print(chunk_list)
print("[INFO] running fslmerge ....\nfslmerge -t [OUTPUT FILENAME][PE FILES]")

with Pool(5) as p:
    p.map(merge_files, chunk_list)
print("[INFO] process complete.")
print('[INFO] find images here: ',concat_path )

[INFO] chunksize: 10
[INFO] running fslmerge ....
fslmerge -t [OUTPUT FILENAME][PE FILES]
[INFO] process complete.
[INFO] find images here:  /projects/niblab/experiments/chocolate_milkshake/data/betaseries/niftis_concat


## Transform Functionals (flirt & fslmaths) <a class='anchor' id='fourth-bullet'></a> 
To use the BigBrain300 rois we need to transform the nifti task files to match the mask.
ROIs location on renci: `/projects/niblab/parcellations/chocolate_decoding_rois`   

Big Brain 300: `/projects/niblab/parcellations/chocolate_decoding_rois/old_rois/bigBrain300_atlas`
  
`flirt`: the main program that performs affine registration. The options we use here:  
* `-in`, an input  
* `-ref`, a reference volume  
* `applyxfm`, `-init` and `-out`, apply a saved transformation to a volume   

  
  
*For these usages the reference volume must still be specified as this sets the voxel and image dimensions of the resulting volume.*



In [199]:
print('[INFO] transform functionals to match the mask.')

reference_nifti='/projects/niblab/parcellations/chocolate_decoding_rois/mni2ace.nii.gz'
reference_mat='/projects/niblab/parcellations/chocolate_decoding_rois/mni2ace.mat'
for nii in fslmerged_files:
    
    # setup and run flirt
    nii=nii.replace('.nii.gz', '')
    out=nii+'_3mm'
    flirt_cmd="flirt -in {} -ref {} -init {} -applyxfm -out {}".format(nii, reference_nifti, reference_mat, out)
    #print('[INFO] flirt command: \n{}'.format(flirt_cmd))
    #os.system(flirt_cmd)
    
    fslmaths_cmd='fslmaths {} -thr 0.9 {}'.format(out,out)
    #print('[INFO] fslmaths command: \n{}'.format(fslmaths_cmd))
    #os.system(fslmaths_cmd)
    
print('[INFO] transformation process complete.')

    
    

[INFO] transform afunctionals to match the mask.
[INFO] transformation process complete.


In [83]:
def tranform_niftis(niftis, keyword=keyword):
    reference_nifti='/projects/niblab/parcellations/chocolate_decoding_rois/mni2ace.nii.gz'
    reference_mat='/projects/niblab/parcellations/chocolate_decoding_rois/mni2ace.mat'
    for nii in niftis:

        # setup and run flirt
        nii=nii.replace('.nii.gz', '')
        out=nii+'_3mm'
        flirt_cmd="flirt -in {} -ref {} -init {} -applyxfm -out {}".format(nii, reference_nifti, reference_mat, out)
        #print('[INFO] flirt command: \n{}'.format(flirt_cmd))
        os.system(flirt_cmd)

        fslmaths_cmd='fslmaths {} -thr 0.9 {}'.format(out,out)
        #print('[INFO] fslmaths command: \n{}'.format(fslmaths_cmd))
        os.system(fslmaths_cmd)

In [84]:

print('[INFO] transform functionals to match the mask.')
chunksize=10
# grab concatenated (fslmerge) data
keyword="LF_HS_minus_h20"
fslmerged_files=glob.glob(os.path.join(concat_path,'%s/*%s*.nii.gz'%(keyword,keyword)))
#fslmerged_files[:4]
print("[INFO] chunksize: {}".format(chunksize))
chunk_list=chunks(fslmerged_files, chunksize)
with Pool(5) as p:
    p.map(tranform_niftis, chunk_list)
print('[INFO] transformation process complete.')


[INFO] transform functionals to match the mask.
[INFO] chunksize: 10
[INFO] transformation process complete.


## Pull ROI timeseries  <a class='anchor' id='fifth-bullet'></a>

Pull individual rois timeseries for each subject. 

Example command:  


    fslmeants -i ~/sub-001_punish.nii.gz -o ~/3_pull_timeseries/sub-001_punish_AI_35_23_-6_asymPREPspace.nii.gz.txt -m ~/AI_35_23_-6_asymPREPspace.nii.gz



Here is the code:

In [19]:


def pull_timeseries(roi_list, bb300_path='/projects/niblab/parcellations/bigbrain300',roi_df='/projects/niblab/parcellations/bigbrain300/renaming.csv'):

    
    bad_subs=[]
    #ICD.display(roi_df)

    # load asymmetrical nifti roi files
    asym_niftis=glob.glob("/projects/niblab/parcellations/bigbrain300/MNI152Asymmetrical_3mm/*.nii.gz")

    # load roi list
    out_dir = os.path.join(beta_path, 'rois/big300')
    #print('[INFO] OUT DIRECTORY: %s \n'%out_dir)

    #roi_df.set_index("final order name", inplace=True)
    #ICD.display(roi_df)#.head())

    # run parallel job pools


    # loop through the roi file list
    #print(roi_list[:3])
    for nifti in sorted(roi_list)[:2]:
        #print('[INFO] loop1')
        #print(nifti)
        subj_id = nifti.split("/")[-1].split("_")[0]
        subj_condition=nifti.split("/")[-1].split(".")[0].replace("_3mm", "")
        #print('[INFO] roi: %s %s'%(subj_id, subj_condition))

        # loop through roi reference list
        for ref_nifti in sorted(asym_niftis):
            #print('[INFO] reference roi: %s'%ref_nifti)
            roi = ref_nifti.split('/')[-1].split(".")[0]
            out_path = os.path.join(out_dir, "{}_{}.txt".format(subj_condition, roi))
            #print(roi, out_path)
            cmd='fslmeants -i {} -o {} -m {}'.format(nifti, out_path, ref_nifti)
            try:
                cmd='fslmeants -i {} -o {} -m {}'.format(nifti, out_path, ref_nifti)
                #print("Running shell command: {}".format(cmd))
                #os.system(cmd)
            except:
                bad_subs.append((subj_id, subj_condition))
        
        #print('[INFO] finished processing for %s'%subj_id)


    return "%s"%bad_subs

   

In [222]:
# load roi
print("[INFO] loading roi and reference file....")

# task rois loaded
task_rois=glob.glob(os.path.join(concat_path,'*%s_3mm.nii.gz'%keyword))
#task_rois[:5]
print("[INFO] {} task roi nifti files being processed.".format(len(task_rois)))

chunksize=10
print("[INFO] chunksize: {}".format(chunksize))
chunk_list=chunks(task_rois, chunksize)
#roi_df['network']
# pull timeseries by rois --fslmeants command


with Pool(10) as p:
    p.map(pull_timeseries, chunk_list)
print("[INFO] process complete.")



[INFO] loading roi and reference file....
[INFO] 227 task roi nifti files being processed.
[INFO] chunksize: 10
[INFO] process complete.


**Submit process as a batch job for large datasets**  

Two files:  
- timeseries_pull.job  
- timeseries_pull.py  

*quick view of file contents below* -- note,for now paths may need to be updated directly in the scripts for your unqiue setup, flexible script in progress.

 

Batch file:

In [20]:
#!cat /projects/niblab/experiments/chocolate_milkshake/data/code/timeseries_pull.job

Python file:

In [21]:
#!cat /projects/niblab/experiments/chocolate_milkshake/data/code/timeseries_pull.py

To submit job file:  

In [22]:
!sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/timeseries_pull.job


Submitted batch job 3050537


To view job:

In [107]:
!squeue --job 3050537

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           3050537     batch timeseri   nbytes  R    3:39:45      1 largemem-0-0


### Get timeseries roi task files 

In [12]:
# view timeseries 
listdir('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/big300')[:5]


['sub-049_task-milkshakeC_bb300_MNI152Asymm3mm_001.txt',
 'sub-001_task-milkshakeB_bb300_MNI152Asymm3mm_001.txt',
 'sub-112_task-milkshakeD_bb300_MNI152Asymm3mm_001.txt',
 'sub-097_task-milkshakeC_bb300_MNI152Asymm3mm_001.txt',
 'sub-020_task-milkshakeC_bb300_MNI152Asymm3mm_001.txt']

In [13]:
# view number of subject timeseries rois created/pulled
len(glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/big300/*.txt'))



67200

## Combine Timeseries into Matrix  <a class="anchor" id="sixth-bullet"></a>
Combine timeseries roi files into, **one file per condition per participant**  

In [14]:
subject_ids=list(data_dict.keys())
subject_ids[-3:]


['sub-150', 'sub-151', 'sub-154']

### Create Matrix

Loop through subject ids and combine in matrix

In [10]:
def create_subject_matrices(subjects):
    for subject_id in subjects:
        tasks=list(data_dict[subject_id].keys())
        for task in tasks:

            # get roi texts for subject / condition
            roi_txts = glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/big300/%s_%s*txt'%(subject_id,task))
            df_lst=[]
            for txt in roi_txts: 
                #print(txt)
                df_temp = pd.read_csv(txt, sep="\n", header=None)
                #print(df_temp)
                df_lst.append(df_temp)
            try:
                df_concat= pd.concat(df_lst, axis=1, sort=False)
                #print(df_concat)

                # write output file 
                outfile='/projects/niblab/experiments/chocolate_milkshake/data/betaseries/subject_matrices/%s_%s-receipt.txt'%(subject_id,task)
                #print('[INFO] making file %s'%outfile)
                #df_concat.to_csv(outfile, header=None, index=None, sep='\t')
            except:
                print('[INFO]error with %s condition %s, passing...'%(subject_id,task))
        



In [29]:
roi_txts = glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/big300/%s_%s*txt'%(subject_id,task))
outfile='/projects/niblab/experiments/chocolate_milkshake/data/betaseries/subject_matrices/%s_%s.txt'%(subject_id,task)

subject_chunks=chunks(subject_ids, 10)
#subject_chunks[:2]

print("[INFO] processing %s lists"%len(subject_chunks))


[INFO] processing 12 lists


Run parallel process:

In [31]:
print("[INFO] making subject condition matrix from rois timeseries...")

with Pool(10) as p:
    p.map(create_subject_matrices, subject_chunks)
    
print("[INFO] completed process.")



[INFO] making subject condition matrix from rois timeseries...
[INFO]error with sub-088 condition task-milkshakeD, passing...
[INFO]error with sub-032 condition task-milkshakeD, passing...
[INFO]error with sub-033 condition task-milkshakeC, passing...
[INFO]error with sub-015 condition task-milkshakeC, passing...
[INFO]error with sub-081 condition task-milkshakeC, passing...
[INFO] completed process.


Quick view of new files:

In [34]:
listdir('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/subject_matrices')[-5:]

['sub-142_task-milkshakeB-receipt.txt',
 'sub-143_task-milkshakeC-receipt.txt',
 'sub-143_task-milkshakeA-receipt.txt',
 'sub-144_task-milkshakeD-receipt.txt',
 'sub-144_task-milkshakeA-receipt.txt']

In [35]:
len(glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/subject_matrices/*.txt'))

224